# Supply & Demand Analysis System
## Business Intelligence Tool for Workforce Capacity Management

\* ***all business specific information has been removed or replaced with generic examples*** *

### Overview
This Jupyter notebook implements an automated system for analyzing and reporting on workforce supply and demand across different business categories. The system helps management make informed decisions about resource allocation, hiring needs, and capacity planning by comparing available workforce capacity against current business demands.

### Business Context
The system tracks two key metrics:
- **Supply**: Available workforce capacity, including:
  - Current workers with additional capacity
  - New hires in training
  - Workers completing certification programs
  - Part-time and full-time staff availability
- **Demand**: Business needs, including:
  - Backfill requests for existing positions
  - New project staffing requirements
  - Additional capacity needs across different business categories

### Technical Implementation
This notebook:
1. **Data Collection**:
   - Pulls workforce data from Salesforce using custom queries
   - Imports capacity base data from Excel files
   - Retrieves historical supply/demand data for trend analysis

2. **Data Processing**:
   - Cleans and standardizes data from multiple sources
   - Calculates key metrics like FTE (Full-Time Equivalent) capacity
   - Segments data by business categories and worker types
   - Handles various employment statuses (full-time, part-time, training)

3. **Analysis**:
   - Compares current supply against demand
   - Calculates surpluses and deficits by category
   - Tracks changes from previous weeks
   - Identifies new capacity additions and filled positions

4. **Reporting**:
   - Generates a comprehensive Excel workbook with multiple sheets:
     - Summary of current supply and demand
     - Core business line analysis
     - Week-over-week changes
     - Detailed capacity changes
     - Demand fluctuations

### Output
The final report provides stakeholders with:
- Current state of workforce capacity
- Areas requiring additional resources
- Historical trends and changes
- Detailed breakdowns by business category
- Training pipeline visibility

This automated system replaces manual tracking processes, providing consistent, accurate, and timely workforce capacity insights for strategic decision-making.

# Imports

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import glob
from simple_salesforce import Salesforce
# Import the module under a specific name
import importlib
import sf_queries_class
importlib.reload(sf_queries_class)
from sf_queries_class import SfQueries
import my_sf_secrets
import capacity_portion
importlib.reload(capacity_portion)
import create_capbase
importlib.reload(create_capbase)
my_sf_username, my_sf_password, my_sf_security_token = my_sf_secrets.get_my_sf_secrets()
queries = SfQueries(
    username=my_sf_username,
    password=my_sf_password,
    security_token=my_sf_security_token
)

In [ ]:
# get the username from the system
username = os.getlogin()

today = datetime.today().strftime('%B %d %Y').replace(" 0", " ")
filename = f"{path_to_planning_teams_folder}CapBase/capbase {today}.xlsx"
print(f'{filename}')

In [ ]:
all_files = glob.glob(f"{path_to_planning_teams_folder}/Supply Demand Summary/supply_demand_analysis_*.xlsx")
if all_files:
    last_weeks_supply_demand = max(all_files, key=os.path.getctime)
    
    # Extract the date from the latest_file_path
    latest_date_str = last_weeks_supply_demand.split('_')[-1].split('.')[0]
    
    # Convert the extracted date to datetime object for comparison
    latest_date = datetime.strptime(latest_date_str, '%B %d %Y')
    
    # Compare with today's date
    if latest_date.strftime('%B %d %Y') == today:
        # Exclude the latest file and take the second newest
        all_files.remove(last_weeks_supply_demand)
        last_weeks_supply_demand = max(all_files, key=os.path.getctime)
    
    print(last_weeks_supply_demand)
else:
    print("No files found.")

# Demand

In [5]:
with open('ops_data_pull_cleaned.sql', 'r') as file:
    sql_query = file.read()
# Now you can use cleaned_sql_query in your function

sql_query_cleaned = sql_query.replace("\n", " ")

In [6]:
ops_plan = queries.convert_salesforce_data_to_df(queries.sf.query_all(query=sql_query_cleaned))
ops_plan = ops_plan.rename(columns={
    'SF_COLUMN_NAME__c': 'RENAMED_COLUMN'
})

# Calculate 'RR_Age_Days'
ops_plan['Created Date'] = pd.to_datetime(ops_plan['Created Date']) 
ops_plan['RR_Age_Days'] = (datetime.now().astimezone() - ops_plan['Created Date']).dt.days
ops_plan['Requested FTE'] = round(ops_plan['Requested Hours'] / 40, 2)

# Min-Max scaling for 'normalized_days'
min_days = ops_plan['RR_Age_Days'].min()
max_days = ops_plan['RR_Age_Days'].max() + 2
ops_plan['normalized_days'] = (ops_plan['RR_Age_Days'] - min_days) / (max_days - min_days)

# Calculate 'composite'
ops_plan['composite'] = ops_plan['Staffing Request Priority Total'] + ops_plan['normalized_days']

# Create 'In Implementation' column
ops_plan['In Implementation'] = ops_plan['Stage'].apply(lambda x: 1 if x in ['In Implementation', 'Delayed'] else 0)

In [9]:
ops_plan_fil = ops_plan[((ops_plan['Status'] == 'Recruit') | (ops_plan['Status'] == 'Draft')) & 
                        ((ops_plan['Team Position'] != 'Backup Team Leader') & (ops_plan['Team Position'] != 'QAR')) & 
                        (ops_plan['Stage'] != 'Hold') & (ops_plan['Stage'] != 'Not Implemented')] 


In [ ]:
for col in ops_plan_fil.columns:
    if 'product' in col.lower():
        print(col)

In [ ]:
ops_plan_fil[(ops_plan_fil['Project Category'] == 'Cancer') & (ops_plan_fil['Team Position'] != 'CDSS')]['Primary Product Skilll: Case Type'].unique()

In [ ]:
# if the Team Position is not CDSS but the product skill contains 'Follow up' or 'Case Finding' then the Team Position should be CDSS
ops_plan_fil.loc[(ops_plan_fil['Project Category'] == 'Cancer') & (ops_plan_fil['Team Position'] != 'CDSS') &
                    ((ops_plan_fil['Primary Product Skilll: Case Type'].str.contains('Follow up')) |
                    (ops_plan_fil['Primary Product Skilll: Case Type'].str.contains('Case Finding'))), 'Team Position'] = 'CDSS'
# replace 'Registry Operations' and 'Temporary Resource' with 'Abstractor'
ops_plan_fil.loc[(ops_plan_fil['Project Category'] == 'Cancer') & ((ops_plan_fil['Team Position'] == 'Registry Operations') | 
                                                                   (ops_plan_fil['Team Position'] == 'Temporary Resource')), 'Team Position'] = 'Abstractor'
# replace any 'Abstractor Depth' with 'Abstractor'
ops_plan_fil.loc[((ops_plan_fil['Team Position'] == 'Abstractor Depth') |
                  (ops_plan_fil['Team Position'] == 'Backup') |
                  (ops_plan_fil['Team Position'] == 'Temporary Resource') |
                  (ops_plan_fil['Team Position'] == 'Institute')), 'Team Position'] = 'Abstractor'

ops_plan_gr = ops_plan_fil.groupby(["Project Category", "Team Position"]).sum(['Requested FTE', 'Requested Hours'])[['Requested FTE', 'Requested Hours']]
ops_plan_gr

In [14]:
ops_plan_gr_index_reset = ops_plan_gr.reset_index()

# Supply

In [ ]:
# import the excel file from the path
try: 
    capbase = pd.read_excel(filename).fillna({'current_skills': ""})
except:
    create_capbase.create_capbase_file(queries)
    capbase = pd.read_excel(filename).fillna({'current_skills': ""})
capbase.head()

In [ ]:
capbase_abstractors_only_1 = capbase[~(capbase['Resource Role'].isin(['Clinical Data Lead', 
                                                                      'Clinical Data Quality Specialist', 
                                                                      'Clinical Services Lead']))].copy()
capbase_abstractors_only_1['Resource Role'].unique()

In [ ]:
capbase_abstractors_only_1.columns

In [18]:

def grouping_fx(df, new_col, date_col):
    # convert 
    df.loc[:, new_col] = (datetime.now() - pd.to_datetime(df[date_col])).dt.days
    
    choices = ["negative", "<30", "30-89", "90-179", "180-364", ">=YEAR"]
    
    df.loc[:, f"{new_col} Group"] = pd.cut(df[new_col], 
                                 bins=[-float('inf'), -1, 29, 89, 179, 364, float('inf')],
                                 labels=choices,
                                 include_lowest=True)
    
    return df
if 'Tenure Group' not in capbase_abstractors_only_1.columns:
    capbase_abstractors_only_gr = grouping_fx(capbase_abstractors_only_1, 'Tenure', 'Resource_Start_Date')

    capbase_Tr = capbase_abstractors_only_gr.copy()
    capbase_Tr['Tr'] = capbase_Tr['Paid Onboarding Program'].isin([
        "Active - Institute",
        "Active - Tr Oncology",
        "Complete - Tr Oncology",
        "Complete - Institute"
    ])

    capbase_abstractors_only = capbase_Tr.copy()
    capbase_abstractors_only['Tenure Group'] = np.select(
        [
            (capbase_abstractors_only['Tr'] & (capbase_abstractors_only['Tenure'] >= 0) & (capbase_abstractors_only['Tenure'] < 112)),
            (capbase_abstractors_only['Tr'] & (capbase_abstractors_only['Tenure'] >= 112))
        ],
        [
            "Tr - Ramp",
            "Tr - Mature"
        ],
        default=capbase_abstractors_only['Tenure Group']
    )
else:
    capbase_abstractors_only = capbase_abstractors_only_1.copy()



In [ ]:
capbase_abstractors_only['Tenure Group'].unique()

In [ ]:
capbase_abstractors_only['Paid Onboarding Program'].unique()

In [ ]:
capbase_abstractors_only.loc[:, 'current_skills'] = np.where((capbase_abstractors_only['Group Name'] == 'GWTG') & 
                                                             (capbase_abstractors_only['Paid Onboarding Program'] == 'Active - Institute') &
                                                              (capbase_abstractors_only['current_skills'].str.contains('Specific Category 1')),
                                                              capbase_abstractors_only['current_skills'].str.replace('Specific Category 1', ''),
                                                              capbase_abstractors_only['current_skills'])
capbase_abstractors_only.loc[:, 'current_skills'] = np.where((capbase_abstractors_only['Group Name'] == 'Specific Category 1') & 
                                                             (capbase_abstractors_only['Paid Onboarding Program'] == 'Active - Institute') &
                                                              (capbase_abstractors_only['current_skills'].str.contains('Specific Category 4')),
                                                              capbase_abstractors_only['current_skills'].str.replace('Specific Category 4', ''),
                                                              capbase_abstractors_only['current_skills'])
# select CBIZ_Name, 'Full Time Status', 'Paid Onboarding Program', 'Tenure Group', 'current_skills', 'Capticket Hours', 'Capacity_Planned' from capbase
capbase_sel = capbase_abstractors_only[['CBIZ_Name', 'Full Time Status', 'Paid Onboarding Program', 
                                        'Tenure Group', 'current_skills', 'Capticket Hours', 'Capacity_Planned', 'Senior Director', 'Group Name']]
# drop duplicates
capbase_sel = capbase_sel.drop_duplicates()
# filter to only those individuals with Capticket Hours > 0 or Capacity_Planned > 0
capbase_sel_capacity = capbase_sel[(capbase_sel['Capticket Hours'] > 0) | 
                          (~(capbase_sel['Paid Onboarding Program'].isin(['Active - Institute', 'Active - Oncology', 'Active - CDSS Oncology NTU'])) &
                            (capbase_sel['Capacity_Planned'] > 0) & (capbase_sel['Tenure Group'].isin(['30-89', 'negative', '<30']))) |
                            ((capbase_sel['Paid Onboarding Program'].isin(['Active - Institute', 'Active - Oncology', 'Active - CDSS Oncology NTU'])) &
                            (capbase_sel['Capacity_Planned'] > 0))]
# if 'current_skills' contains 'CM' but 'Senior Director' is not 'Paul Gasque', then replace 'CM' with ''
capbase_sel_capacity.loc[:, 'current_skills'] = np.where(((capbase_sel_capacity['current_skills'].str.contains('Specific Category 1')) &
                                                    (capbase_sel_capacity['Senior Director'] != 'SD NAME')) |
                                                    ((capbase_sel_capacity['current_skills'].str.contains('Specific Category 1')) &
                                                     ((capbase_sel_capacity['current_skills'].str.contains('Specific Category 2')) |
                                                       (capbase_sel_capacity['current_skills'].str.contains('Specific Category 3')))), 
                                                       capbase_sel_capacity['current_skills'].str.replace('Specific Category 1', ''), 
                                                       capbase_sel_capacity['current_skills'])
# if 'current_skills' is 'STS' then change it to 'STS-ACS'
capbase_sel_capacity.loc[:, 'current_skills'] = np.where(capbase_sel_capacity['current_skills'].str.contains('Specific Category 4,'), capbase_sel_capacity['current_skills'].str.replace('Specific Category 4,', ''),
                                                          capbase_sel_capacity['current_skills'])
capbase_sel_capacity


In [23]:
capbase_sel_capacity.loc[:, 'current_skills'] = capbase_sel_capacity['current_skills'].apply(lambda x: [item.strip() for item in str(x).split(',') if (item != ' ') & (item != '')])

In [ ]:
# create a new column for current_skills_length which is the length of the list in the current row of the 'current_skills' column
capbase_sel_capacity.loc[:, 'current_skills_length'] = capbase_sel_capacity['current_skills'].apply(lambda x: len(x)).copy()
# create a new column 'capacity' which is Capticket Hours unless it is null than fill it in with Capacity_Planned
capbase_sel_capacity.loc[:, 'capacity'] = np.where(capbase_sel_capacity['Capticket Hours'].isnull(), (capbase_sel_capacity['Capacity_Planned'])/capbase_sel_capacity['current_skills_length'], 
                                                   (capbase_sel_capacity['Capticket Hours'])/capbase_sel_capacity['current_skills_length'])
capbase_sel_capacity.loc[:, 'ticket_capacity'] = capbase_sel_capacity['Capticket Hours']/capbase_sel_capacity['current_skills_length']


In [25]:
# explode out the current_skills column for each of the lists in the current_skills column into multiple rows, one for each item in the list
capbase_sel_capacity_exploded = capbase_sel_capacity.explode('current_skills')

In [ ]:
capbase_sel_capacity_exploded.groupby('current_skills').sum()[['capacity']]

In [ ]:
# fill any 'Paid Onboarding Program' that is null with 'None'
capbase_sel_capacity_exploded.loc[:, 'Paid Onboarding Program'] = capbase_sel_capacity_exploded['Paid Onboarding Program'].fillna('None')
# if 'Paid Onboarding Program' contains 'Complete' then replace with 'Complete'
capbase_sel_capacity_exploded.loc[:, 'Paid Onboarding Program'] = np.where(capbase_sel_capacity_exploded['Paid Onboarding Program'].str.contains('Complete'), 'Complete', capbase_sel_capacity_exploded['Paid Onboarding Program'])
# if 'Paid Onboarding Program' is 'None' then replace with 'Complete'
capbase_sel_capacity_exploded.loc[:, 'Paid Onboarding Program'] = np.where(capbase_sel_capacity_exploded['Paid Onboarding Program'] == 'None', 'Complete', capbase_sel_capacity_exploded['Paid Onboarding Program'])
# if 'Paid Onboarding Program' contains 'Active' and 'Tr' then replace with 'Active - Institute'
capbase_sel_capacity_exploded.loc[:, 'Paid Onboarding Program'] = np.where((capbase_sel_capacity_exploded['Paid Onboarding Program'].str.contains('Active')) &
                                                                         (capbase_sel_capacity_exploded['Paid Onboarding Program'].str.contains('Tr')), 'Active - Institute', capbase_sel_capacity_exploded['Paid Onboarding Program'])
# create a column called 'New Hires' which is if 'Tenure Group' is in ['30-89', 'negative', '<30']
capbase_sel_capacity_exploded.loc[:, 'New Hires'] = np.where(capbase_sel_capacity_exploded['Tenure Group'].isin(['30-89', 'negative', '<30']), 'New Hires', '')
capbase_sel_capacity_exploded.loc[:, 'Tr Hrs'] = np.where((capbase_sel_capacity_exploded['Tenure Group'] == 'Tr - Ramp') |
                                                           ((capbase_sel_capacity_exploded['Tenure Group'].isin(['30-89', 'negative', '<30'])) & 
                                                             (capbase_sel_capacity_exploded['Paid Onboarding Program'] == 'Active - Institute')), 'Training', 
                                                           np.where((capbase_sel_capacity_exploded['Tenure Group'] == 'QCI - Mature') & 
                                                                    (capbase_sel_capacity_exploded['Paid Onboarding Program'] == 'Active - Institute'), 'Est', ''))

capbase_sel_capacity_exploded_gr = capbase_sel_capacity_exploded.groupby(['current_skills', 'Full Time Status', 'Paid Onboarding Program', 'New Hires', 'Tr Hrs']).sum()[['capacity', 'Capticket Hours', 'ticket_capacity']].round(1)
# add a column to capbase_sel_capacity_exploded_gr that is 'FTE capacity' which is 'capacity' divided by 40
capbase_sel_capacity_exploded_gr.loc[:, 'FTE capacity'] = round(capbase_sel_capacity_exploded_gr['capacity']/40, 2)
capbase_sel_capacity_exploded_gr

In [ ]:
writer2 = pd.ExcelWriter(f"{path_to_planning_teams_folder}/CapBase/supply_demand_raw_data_{today}.xlsx", engine='xlsxwriter')
# drop the 'Tenure_Group', 'Paid Onboarding Program' columns from capbase_sel_capacity_exploded
capbase_sel_capacity_exploded_drp = capbase_sel_capacity_exploded.drop(['Tenure Group', 'Paid Onboarding Program'], axis = 1)
capbase_sel_capacity_exploded_drp.loc[:, 'Run_Date'] = today
# place Run_Date in the first column
capbase_sel_capacity_exploded_drp = capbase_sel_capacity_exploded_drp[['Run_Date'] + list(capbase_sel_capacity_exploded_drp.columns[:-1])]
capbase_sel_capacity_exploded_drp.to_excel(writer2, sheet_name = 'Supply', index = False)
ops_plan_sel = ops_plan_fil.loc[:, ['Team Name', "Project Category", "Team Position",'Requested FTE', 'Requested Hours']]
ops_plan_sel_gr0 = ops_plan_sel.loc[ops_plan_sel['Requested FTE'] > 0, :].copy()
ops_plan_sel_gr0.loc[:, 'Run_Date'] = today
ops_plan_sel_gr0 = ops_plan_sel_gr0.loc[:, ['Run_Date'] + list(ops_plan_sel_gr0.columns[:-1])]
ops_plan_sel_gr0.to_excel(writer2, sheet_name = 'Demand', index = False)
print(f"{path_to_planning_teams_folder}/CapBase/supply_demand_raw_data_{today}.xlsx")
writer2.close()

In [ ]:
# filter ops_plan_gr_index_reset if Requested FTE > 0
ops_plan_gr_index_reset_gr0 = ops_plan_gr_index_reset[ops_plan_gr_index_reset['Requested FTE'] > 0]
# if the project Category == 'Cancer' then concatenate it with ' - ' and the 'Team Position'
ops_plan_gr_index_reset_gr0.loc[:, 'Project Category'] = np.where(ops_plan_gr_index_reset_gr0['Project Category'] == 'Cancer', ops_plan_gr_index_reset_gr0['Project Category'] + ' - ' + ops_plan_gr_index_reset_gr0['Team Position'], ops_plan_gr_index_reset_gr0['Project Category'])
ops_plan_gr_index_reset_gr0

In [ ]:
capbase_sel_capacity_exploded_gr_index_reset = capbase_sel_capacity_exploded_gr.reset_index()
capbase_sel_capacity_exploded_gr_index_reset.loc[:, 'current_skills'] = \
    np.where(((capbase_sel_capacity_exploded_gr_index_reset['current_skills'] == 'Cancer') & 
             (capbase_sel_capacity_exploded_gr_index_reset['Paid Onboarding Program'] == 'Active - Institute')), 'Cancer - CDSS', 
             np.where(((capbase_sel_capacity_exploded_gr_index_reset['current_skills'] == 'Cancer') & 
             ~(capbase_sel_capacity_exploded_gr_index_reset['Paid Onboarding Program'] == 'Active - Institute')), 'Cancer - Abstractor', capbase_sel_capacity_exploded_gr_index_reset['current_skills']))
capbase_sel_capacity_exploded_gr_index_reset_gr = capbase_sel_capacity_exploded_gr_index_reset.groupby('current_skills').sum()[['FTE capacity']]
capbase_sel_capacity_exploded_gr_index_reset_gr2 = capbase_sel_capacity_exploded_gr_index_reset_gr.reset_index()
capbase_sel_capacity_exploded_gr_index_reset_gr2

## Merging DataFrames

In [ ]:
# just what we need for supply demand analysis
capbase_sel_capacity_exploded_gr_index_reset_gr = capbase_sel_capacity_exploded_gr_index_reset.groupby('current_skills').sum()[['FTE capacity']]
capbase_sel_capacity_exploded_gr_index_reset_gr2 = capbase_sel_capacity_exploded_gr_index_reset_gr.reset_index()

# only what we need from ops plan
ops_plan_gr_index_reset_gr0_sel = ops_plan_gr_index_reset_gr0[['Project Category','Requested FTE']]

supply_demand = capbase_sel_capacity_exploded_gr_index_reset_gr2.merge(ops_plan_gr_index_reset_gr0_sel, left_on='current_skills', right_on='Project Category', how='outer')
# grab the first non-nan value from current_skills and Project Category and put them into current_skills
supply_demand.loc[:, 'current_skills'] = supply_demand['current_skills'].fillna(supply_demand['Project Category'])
# drop the project category column
supply_demand = supply_demand.drop(columns=['Project Category'])
# rename current_skills to 'Project Category'
supply_demand = supply_demand.rename(columns={'current_skills': 'Product Category'})
# replace the NaN's in Requested FTE with 0
# supply_demand.loc[:, 'Requested FTE'] = supply_demand['Requested FTE'].fillna(0)
# create a column for the difference between FTE capacity and Requested FTE if either are null then fill with 0 to get the surplus or deficit
supply_demand.loc[:, 'FTE capacity_null_filled'] = supply_demand['FTE capacity'].fillna(0)
supply_demand.loc[:, 'Requested FTE_null_filled'] = supply_demand['Requested FTE'].fillna(0)
supply_demand.loc[:, 'Surplus/(Deficit)'] = supply_demand['FTE capacity_null_filled'] - supply_demand['Requested FTE_null_filled']
# drop the columns that were used to calculate the surplus/deficit
supply_demand = supply_demand.drop(columns=['FTE capacity_null_filled', 'Requested FTE_null_filled'])
supply_demand = supply_demand.sort_values('Product Category')
supply_demand

In [ ]:
# in capbase_sel_capacity_exploded_gr_index_reset create a new column that concatenates 'Full Time Status' and  'Paid Onboarding Program'  with a ' - '
capbase_sel_capacity_exploded_gr_index_reset.loc[:, 'POB'] = capbase_sel_capacity_exploded_gr_index_reset['Full Time Status'] + ' - ' + capbase_sel_capacity_exploded_gr_index_reset['Paid Onboarding Program']
capbase_sel_capacity_exploded_gr_index_reset

## Supply Break Down

In [36]:
capbase_sel_capacity_exploded_gr_index_reset_cols_dropped = capbase_sel_capacity_exploded_gr_index_reset.drop(columns=['New Hires', 'Tr Hrs', 'Capticket Hours'])
# drop duplicate rows
capbase_sel_capacity_exploded_gr_index_reset_cols_dropped = capbase_sel_capacity_exploded_gr_index_reset_cols_dropped.groupby(['current_skills', 'POB']).sum()[['FTE capacity']]
capbase_sel_capacity_exploded_gr_index_reset_cols_dropped_index_reset = capbase_sel_capacity_exploded_gr_index_reset_cols_dropped.reset_index()

In [ ]:
# create a new dataframe that has current_skills on the x-axis then creates a new column for each unique value in the 'POB' column and for the values uses the 'FTE capacity' column
capbase_sel_capacity_exploded_gr_index_reset_pivot = capbase_sel_capacity_exploded_gr_index_reset_cols_dropped_index_reset.pivot(index='current_skills', columns='POB', values='FTE capacity')
# if any of these columns are missing: FT - Active	FT - Active - QC Institute	FT - Complete	PT - Complete then create a column of nans
for col in ['FT - Active', 'FT - Active - Institute', 'FT - Complete', 'PT - Complete']:
    if col not in capbase_sel_capacity_exploded_gr_index_reset_pivot.columns:
        capbase_sel_capacity_exploded_gr_index_reset_pivot[col] = np.nan
# capbase_sel_capacity_exploded_gr_index_reset_pivot = capbase_sel_capacity_exploded_gr_index_reset_pivot.fillna(0)
capbase_sel_capacity_exploded_gr_index_reset_pivot

In [ ]:
# set the index supply_demand to 'Product Category'
supply_demand_pr_index = supply_demand.set_index('Product Category')
# merge the supply_demand_pr_index with capbase_sel_capacity_exploded_gr_index_reset_pivot on the index
supply_demand_pr_index_merged = supply_demand_pr_index.merge(capbase_sel_capacity_exploded_gr_index_reset_pivot, left_index=True, right_index=True, how='left')
supply_demand_pr_index_merged_fil = supply_demand_pr_index_merged.loc[['Cat 1', 'Cat 2', 'Cat3'], :]
supply_demand_pr_index_merged

## Capacity Tickets

In [39]:
# capbase_sel_capacity_exploded_gr_index_reset_cols_dropped = capbase_sel_capacity_exploded_gr_index_reset.drop(columns=['New Hires', 'QCI Hrs', 'Capticket Hours'])
# drop duplicate rows
capbase_sel_capacity_exploded_gr_index_reset_cols_tickets = capbase_sel_capacity_exploded_gr_index_reset.groupby(['current_skills', 'POB']).sum()[['ticket_capacity']]
capbase_sel_capacity_exploded_gr_index_reset_cols_tickets_index_reset = capbase_sel_capacity_exploded_gr_index_reset_cols_tickets.reset_index()

In [ ]:
capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets = capbase_sel_capacity_exploded_gr_index_reset_cols_tickets_index_reset.pivot(index='current_skills', columns='POB', values='ticket_capacity')
# add '_tickets' to all the column names 
capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets.columns = [col + '_tickets' for col in capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets.columns]
# if any of these columns are missing: FT - Active_tickets	FT - Active - QC Institute_tickets	FT - Complete_tickets	PT - Complete_tickets then create a column of nans
for col in ['FT - Active_tickets', 'FT - Active - QC Institute_tickets', 'FT - Complete_tickets', 'PT - Complete_tickets']:
    if col not in capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets.columns:
        capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets[col] = np.nan
capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets

In [ ]:
# merge the supply_demand_pr_index with capbase_sel_capacity_exploded_gr_index_reset_pivot on the index
supply_demand_pr_index_merged_tickets = supply_demand_pr_index_merged.merge(capbase_sel_capacity_exploded_gr_index_reset_pivot_tickets, left_index=True, right_index=True, how='left')
supply_demand_pr_index_merged_tickets

## QCI Hours

In [ ]:
# filter capbase_sel_capacity_exploded_gr_index_reset to where 'QCI Hrs' doesn't equal ''
capbase_sel_capacity_exploded_gr_index_reset_QCI_hrs = capbase_sel_capacity_exploded_gr_index_reset[capbase_sel_capacity_exploded_gr_index_reset['QCI Hrs'] != '']
capbase_sel_capacity_exploded_gr_index_reset_cols_QCI_hrs = capbase_sel_capacity_exploded_gr_index_reset_QCI_hrs.groupby(['current_skills', 'QCI Hrs']).sum()[['capacity']]
capbase_sel_capacity_exploded_gr_index_reset_cols_QCI_hrs_index_reset = capbase_sel_capacity_exploded_gr_index_reset_cols_QCI_hrs.reset_index()
capbase_sel_capacity_exploded_gr_index_reset_cols_QCI_hrs_index_reset
capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs = capbase_sel_capacity_exploded_gr_index_reset_cols_QCI_hrs_index_reset.pivot(index='current_skills', columns='QCI Hrs', values='capacity')
# # add '_QCI_hrs' to all the column names 
capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs.columns = [col + '_QCI_hrs' for col in capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs.columns]
# if Training_QCI_hrs doesn't exist, add a column of NaNs
if 'Training_QCI_hrs' not in capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs.columns:
    capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs['Training_QCI_hrs'] = np.nan
if 'Est_QCI_hrs' not in capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs.columns:
    capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs['Est_QCI_hrs'] = np.nan
capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs

In [ ]:
supply_demand_pr_index_merged_QCI_hrs = supply_demand_pr_index_merged_tickets.merge(capbase_sel_capacity_exploded_gr_index_reset_pivot_QCI_hrs, left_index=True, right_index=True, how='left')
supply_demand_pr_index_merged_QCI_hrs

## New Hires

In [ ]:
# filter capbase_sel_capacity_exploded_gr_index_reset to where 'QCI Hrs' doesn't equal ''
capbase_sel_capacity_exploded_gr_index_reset_new_hires = capbase_sel_capacity_exploded_gr_index_reset.loc[capbase_sel_capacity_exploded_gr_index_reset['New Hires'] != '']
capbase_sel_capacity_exploded_gr_index_reset_new_hires.loc[:, 'Status - New Hire'] = capbase_sel_capacity_exploded_gr_index_reset_new_hires['Full Time Status'] + ' - ' + capbase_sel_capacity_exploded_gr_index_reset_new_hires['New Hires']
capbase_sel_capacity_exploded_gr_index_reset_cols_new_hires = capbase_sel_capacity_exploded_gr_index_reset_new_hires.groupby(['current_skills', 'Status - New Hire']).sum()[['capacity']]
capbase_sel_capacity_exploded_gr_index_reset_cols_new_hires_index_reset = capbase_sel_capacity_exploded_gr_index_reset_cols_new_hires.reset_index()
capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires = capbase_sel_capacity_exploded_gr_index_reset_cols_new_hires_index_reset.pivot(index='current_skills', columns='Status - New Hire', values='capacity')
if 'FT - New Hires' not in capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires.columns:
    capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires['FT - New Hires'] = np.nan
if 'PT - New Hires' not in capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires.columns:
    capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires['PT - New Hires'] = np.nan
capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires

In [ ]:
supply_demand_pr_index_merged_new_hires = supply_demand_pr_index_merged_QCI_hrs.merge(capbase_sel_capacity_exploded_gr_index_reset_pivot_new_hires, left_index=True, right_index=True, how='left')
supply_demand_pr_index_merged_new_hires_no_multi = supply_demand_pr_index_merged_new_hires.copy()
supply_demand_pr_index_merged_new_hires

In [ ]:
supply_demand_pr_index_merged_new_hires.columns

In [ ]:
# Assuming supply_demand_pr_index_merged_new_hires is already defined and loaded

# Define the labels for the primary column index
primary_column_labels = ['Overall', 'Supply Breakdown', 'Tickets', 'Tr Hrs', 'New Hires']
column_labels_full = []
for label, num_cols in zip(primary_column_labels, [3, 4, 4, 2, 2]):
    labels_i = [label] * num_cols
    column_labels_full.extend(labels_i)

# Create a MultiIndex for the columns
multiindex = pd.MultiIndex.from_arrays([column_labels_full, supply_demand_pr_index_merged_new_hires.columns])

# Assign the new MultiIndex to the DataFrame
supply_demand_pr_index_merged_new_hires.columns = multiindex
supply_demand_pr_index_merged_new_hires
# Reset the index to make the primary column index part of the DataFrame's columns again
# supply_demand_pr_index_merged_new_hires.reset_index(drop=True, inplace=True)

In [48]:
# export to excel
# supply_demand_pr_index_merged_new_hires.to_excel(f"{path_to_planning_teams_folder}supply_demand_analysis_{today}.xlsx", index=True, sheet_name = 'Summary')

# Compare to last week's Supply Demand Summary

In [ ]:
# import last_weeks_supply_demand_summary and skip the first row and set the index to 'Unnamed: 0' 
last_weeks_supply_demand_summary = pd.read_excel(last_weeks_supply_demand, sheet_name='Summary', skiprows=1, index_col='Unnamed: 0')
# remove the first row
last_weeks_supply_demand_summary = last_weeks_supply_demand_summary.iloc[1:]
# last_weeks_supply_demand_summary.columns = multiindex
# fill all the nans with 0
last_weeks_supply_demand_summary = last_weeks_supply_demand_summary.fillna(0)
# add '_last_week' to all the columns
last_weeks_supply_demand_summary.columns = [col + '_last_week' for col in last_weeks_supply_demand_summary.columns]
last_weeks_supply_demand_summary

In [ ]:
# fill all the nans in supply_demand_pr_index_merged_new_hires with 0
supply_demand_pr_index_merged_new_hires_no_multi = supply_demand_pr_index_merged_new_hires_no_multi.fillna(0)
supply_demand_pr_index_merged_new_hires_no_multi

In [ ]:
# do an outer join of supply_demand_pr_index_merged_new_hires_no_multi and last_weeks_supply_demand_summary
supply_demand_pr_index_merged_new_hires_no_multi_last_week = supply_demand_pr_index_merged_new_hires_no_multi.merge(last_weeks_supply_demand_summary, left_index=True, right_index=True, how='outer', suffixes=('', '_last_week'))

# Create 'diff' columns only for columns that exist in both current and previous week
for col in supply_demand_pr_index_merged_new_hires_no_multi.columns:
    if f'{col}_last_week' in supply_demand_pr_index_merged_new_hires_no_multi_last_week.columns:
        supply_demand_pr_index_merged_new_hires_no_multi_last_week[f'{col}_diff'] = supply_demand_pr_index_merged_new_hires_no_multi_last_week[col] - supply_demand_pr_index_merged_new_hires_no_multi_last_week[f'{col}_last_week']

# Order the columns
cols = []
for col in supply_demand_pr_index_merged_new_hires_no_multi.columns:
    cols.append(col)
    if f'{col}_last_week' in supply_demand_pr_index_merged_new_hires_no_multi_last_week.columns:
        cols.extend([f'{col}_last_week', f'{col}_diff'])

supply_demand_pr_index_merged_new_hires_no_multi_last_week = supply_demand_pr_index_merged_new_hires_no_multi_last_week[cols]

# Replace 0s with NaNs
supply_demand_pr_index_merged_new_hires_no_multi_last_week = supply_demand_pr_index_merged_new_hires_no_multi_last_week.replace(0, np.nan)

supply_demand_pr_index_merged_new_hires_no_multi_last_week


In [52]:
# bring in the raw data from last week
supply_last_week = pd.read_excel(f"{path_to_planning_teams_folder}/CapBase/supply_demand_raw_data_{latest_date_str}.xlsx", sheet_name='Supply')
demand_last_week = pd.read_excel(f"{path_to_planning_teams_folder}/CapBase/supply_demand_raw_data_{latest_date_str}.xlsx", sheet_name='Demand')

In [ ]:
# concat supply_last_week to the bottom of capbase_sel_capacity_exploded_drp
supply_raw_both_weeks = pd.concat([capbase_sel_capacity_exploded_drp, supply_last_week])
# replace '' with np.nan
supply_raw_both_weeks = supply_raw_both_weeks.replace('', np.nan)
print(len(supply_raw_both_weeks))
# replace any nans with 0
supply_raw_both_weeks = supply_raw_both_weeks.fillna(0)
# group by 'CBIZ_Name', 'Full Time Status', 'current_skills', 'Capticket Hours', 'capacity'
supply_raw_both_weeks_gr = supply_raw_both_weeks.groupby(['CBIZ_Name', 'Full Time Status', 'current_skills', 'Capticket Hours', 'capacity']).count()['Run_Date'].reset_index()
supply_raw_both_weeks_gr = supply_raw_both_weeks_gr.rename(columns={'Run_Date': 'count'})
supply_raw_both_weeks_gr.loc[:, 'Changed'] = np.where(supply_raw_both_weeks_gr['count'] > 1, 'No', 'Yes')
# drop count column
supply_raw_both_weeks_gr = supply_raw_both_weeks_gr.drop(columns=['count'])
# join this back to the supply_raw_both_weeks dataframe
supply_raw_both_weeks_joined = supply_raw_both_weeks.merge(supply_raw_both_weeks_gr, on=['CBIZ_Name', 'Full Time Status', 'current_skills', 'Capticket Hours', 'capacity'], how='left')
supply_raw_both_weeks_joined
# supply_raw_both_weeks = supply_raw_both_weeks.drop_duplicates(subset=['CBIZ_Name', 'Full Time Status', 'current_skills', 'Capticket Hours', 'capacity'])
# supply_raw_both_weeks


In [ ]:
# get the distinct values in the 'Run_Date' and 'CBIZ_Name' columns
dups_dropped_cbiz = supply_raw_both_weeks_joined.drop_duplicates(subset=['Run_Date', 'CBIZ_Name'])
dups_dropped_cbiz_gr = dups_dropped_cbiz.groupby(['CBIZ_Name']).count()['Run_Date'].reset_index().rename(columns={'Run_Date': 'count'})
dups_dropped_cbiz_gr.loc[:, 'both_weeks'] = np.where(dups_dropped_cbiz_gr['count'] > 1, 'Yes', 'No')
dups_dropped_cbiz_gr = dups_dropped_cbiz_gr.drop(columns=['count'])
# join back to supply_raw_both_weeks_joined on 'CBIZ_Name' 
supply_raw_both_weeks_joined_2 = supply_raw_both_weeks_joined.merge(dups_dropped_cbiz_gr, on=['CBIZ_Name'], how='left')
# if both_weeks is 'No' and 'Run_Date' == last_week then 'Capacity Filled' however, if both_weeks is 'No' and 'Run_Date' == today then 'New Capacity' make this as a new column 'Abstractor Changed'
supply_raw_both_weeks_joined_2.loc[:, 'Abstractor Changed'] = np.where((supply_raw_both_weeks_joined_2['both_weeks'] == 'No') & (supply_raw_both_weeks_joined_2['Run_Date'] == latest_date_str), 'Capacity Filled', 
                                                                       np.where((supply_raw_both_weeks_joined_2['both_weeks'] == 'No') & (supply_raw_both_weeks_joined_2['Run_Date'] == today), 'New Capacity', ''))
# rename current_skills to 'Product Skill Category'
supply_raw_both_weeks_joined_2 = supply_raw_both_weeks_joined_2.rename(columns={'current_skills': 'Product Skill Category'})
# replace any 0s with np.nan
supply_raw_both_weeks_joined_2 = supply_raw_both_weeks_joined_2.replace(0, np.nan)
supply_raw_both_weeks_joined_2


In [55]:
# supply_raw_both_weeks_joined_2[supply_raw_both_weeks_joined_2['Product Skill Category'] == 'PCI']

In [ ]:
# concat supply_last_week to the bottom of capbase_sel_capacity_exploded_drp
demand_raw_both_weeks = pd.concat([ops_plan_sel_gr0, demand_last_week])
demand_raw_both_weeks = demand_raw_both_weeks.replace('', np.nan)
demand_raw_both_weeks.loc[:, 'Project Category'] = np.where(demand_raw_both_weeks['Project Category'] == 'Cancer', demand_raw_both_weeks['Project Category'] + ' - ' + demand_raw_both_weeks['Team Position'], demand_raw_both_weeks['Project Category'])
# drop "Team Position"
demand_raw_both_weeks = demand_raw_both_weeks.drop(columns=['Team Position'])
print(len(demand_raw_both_weeks))
demand_raw_both_weeks_gr = demand_raw_both_weeks.groupby(['Q-Centrix Team Name', 'Project Category', 'Requested Hours']).count()['Run_Date'].reset_index()
demand_raw_both_weeks_gr = demand_raw_both_weeks_gr.rename(columns={'Run_Date': 'count'})
demand_raw_both_weeks_gr.loc[:, 'Changed'] = np.where(demand_raw_both_weeks_gr['count'] > 1, 'No', 'Yes')
# drop count column
demand_raw_both_weeks_gr = demand_raw_both_weeks_gr.drop(columns=['count'])
# join this back to the demand_raw_both_weeks dataframe
demand_raw_both_weeks_joined = demand_raw_both_weeks.merge(demand_raw_both_weeks_gr, on=['Q-Centrix Team Name', 'Project Category', 'Requested Hours'], how='left')
demand_raw_both_weeks_joined

In [ ]:
# get the distinct values in the 'Run_Date' and 'CBIZ_Name' columns
dups_dropped_team_name = demand_raw_both_weeks_joined.drop_duplicates(subset=['Run_Date', 'Q-Centrix Team Name'])
dups_dropped_team_name_gr = dups_dropped_team_name.groupby(['Q-Centrix Team Name']).count()['Run_Date'].reset_index().rename(columns={'Run_Date': 'count'})
dups_dropped_team_name_gr.loc[:, 'both_weeks'] = np.where(dups_dropped_team_name_gr['count'] > 1, 'Yes', 'No')
dups_dropped_team_name_gr = dups_dropped_team_name_gr.drop(columns=['count'])
# join back to demand_raw_both_weeks_joined on 'CBIZ_Name' 
demand_raw_both_weeks_joined_2 = demand_raw_both_weeks_joined.merge(dups_dropped_team_name_gr, on=['Q-Centrix Team Name'], how='left')
# if both_weeks is 'No' and 'Run_Date' == last_week then 'Capacity Filled' however, if both_weeks is 'No' and 'Run_Date' == today then 'New Capacity' make this as a new column 'Abstractor Changed'
demand_raw_both_weeks_joined_2.loc[:, 'Demand Changed'] = np.where((demand_raw_both_weeks_joined_2['both_weeks'] == 'No') & (demand_raw_both_weeks_joined_2['Run_Date'] == latest_date_str), 'Demand Filled', 
                                                                       np.where((demand_raw_both_weeks_joined_2['both_weeks'] == 'No') & (demand_raw_both_weeks_joined_2['Run_Date'] == today), 'New Demand', ''))
# replace any 0s with np.nan
demand_raw_both_weeks_joined_2 = demand_raw_both_weeks_joined_2.replace(0, np.nan)
demand_raw_both_weeks_joined_2

In [58]:
writer = pd.ExcelWriter(f"{path_to_planning_teams_folder}Supply Demand Summary/supply_demand_analysis_{today}.xlsx", engine='xlsxwriter')
supply_demand_pr_index_merged_fil.to_excel(writer, sheet_name = 'Summary Core Lines')
supply_demand_pr_index_merged_new_hires.to_excel(writer, sheet_name = 'Summary')
supply_demand_pr_index_merged_new_hires_no_multi_last_week.to_excel(writer, sheet_name='Changes Since Last Week')
supply_raw_both_weeks_joined_2.to_excel(writer, sheet_name='Capacity Changes', index=False)
demand_raw_both_weeks_joined_2.to_excel(writer, sheet_name='Demand Changes', index=False)
writer.close()

In [59]:
# # After writing the data to Excel, open the workbook and set the column widths
import openpyxl

# Load the workbook
workbook = openpyxl.load_workbook(f"{path_to_planning_teams_folder}Supply Demand Summary/supply_demand_analysis_{today}.xlsx")

# get all of the sheet names 
sheet_names = workbook.sheetnames
for sheet_name in sheet_names:
    # Select the sheet
    sheet = workbook[sheet_name]
    # print(sheet_name)
    if sheet_name == 'Summary':
        for column in sheet.columns:
            max_length = 0
            column_letter = column[1].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            sheet.column_dimensions[column_letter].width = adjusted_width
    else:
        # Set the column widths
        for column in sheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            sheet.column_dimensions[column_letter].width = adjusted_width

# # Save the workbook
workbook.save(f"{path_to_planning_teams_folder}Supply Demand Summary/supply_demand_analysis_{today}.xlsx")
workbook.close()